<a href="https://colab.research.google.com/github/JackStrabala/mgmt467-analytics-portfolio/blob/main/Unit2_Lab1_PromptPlusExample_Colab_Kaggle_GCS_BQ_DQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MGMT 467 — Prompt-Driven Lab (with Commented Examples)
## Kaggle ➜ Google Cloud Storage ➜ BigQuery ➜ Data Quality (DQ)

**How to use this notebook**
- Each section gives you a **Build Prompt** to paste into Gemini/Vertex AI (or Gemini in Colab).
- Below each prompt, you’ll see a **commented example** of what a good LLM answer might look like.
- **Do not** just uncomment and run. Use the prompt to generate your own code, then compare to the example.
- After every step, run the **Verification Prompt**, and write the **Reflection** in Markdown.

> Goal today: Download the Netflix dataset (Kaggle) → Stage on GCS → Load into BigQuery → Run DQ profiling (missingness, duplicates, outliers, anomaly flags).


### Academic integrity & LLM usage
- Use the prompts here to generate your own code cells.
- Read concept notes and write the reflection answers in your own words.
- Keep credentials out of code. Upload `kaggle.json` when asked.


## Learning objectives
1) Explain **why** we stage data in GCS and load it to BigQuery.  
2) Build an **idempotent**, auditable pipeline.  
3) Diagnose **missingness**, **duplicates**, and **outliers** and justify cleaning choices.  
4) Connect DQ decisions to **business/ML impact**.


## 0) Environment setup — What & Why
Authenticate Colab to Google Cloud so we can use `gcloud`, GCS, and BigQuery. Set **PROJECT_ID** and **REGION** once for consistency (cost/latency).

### Build Prompt (paste to LLM)
You are my cloud TA. Generate a single **Colab code cell** that:
1) Authenticates to Google Cloud in Colab,  
2) Prompts for `PROJECT_ID` via `input()` and sets `REGION="us-central1"` (editable),  
3) Exports `GOOGLE_CLOUD_PROJECT`,  
4) Runs `gcloud config set project $GOOGLE_CLOUD_PROJECT`,  
5) Prints both values. Add 2–3 comments explaining what/why.
End with a comment: `# Done: Auth + Project/Region set`.


In [ ]:
from google.colab import files
import os

# Prompt the user to upload their kaggle.json file.
# This file contains your Kaggle API credentials.
print("Upload your kaggle.json (Kaggle > Account > Create New API Token)")
uploaded = files.upload()

# Create the .kaggle directory if it doesn't exist.
# This is where Kaggle expects to find the credentials file.
os.makedirs('/root/.kaggle', exist_ok=True)

# Save the uploaded file to the correct location.
# Using the first uploaded file as we expect only one (kaggle.json).
with open('/root/.kaggle/kaggle.json', 'wb') as f:
    f.write(uploaded[list(uploaded.keys())[0]])

# Set file permissions to 0600 (owner read/write only).
# This is crucial for security to prevent other users from accessing your API key.
os.chmod('/root/.kaggle/kaggle.json', 0o600)

# Verify the Kaggle installation by printing the version.
# This confirms the CLI is installed and can access the credentials.
!kaggle --version

# Done: Kaggle setup

Upload your kaggle.json (Kaggle > Account > Create New API Token)


Saving kaggle.json to kaggle.json
Kaggle API 1.7.4.5


### Build Prompt (paste to LLM)
You are my cloud TA. Generate a single **Colab code cell** that:
1) Authenticates to Google Cloud in Colab,  
2) Prompts for `PROJECT_ID` via `input()` and sets `REGION="us-central1"` (editable),  
3) Exports `GOOGLE_CLOUD_PROJECT`,  
4) Runs `gcloud config set project $GOOGLE_CLOUD_PROJECT`,  
5) Prints both values. Add 2–3 comments explaining what/why.
End with a comment: `# Done: Auth + Project/Region set`.


In [ ]:
from google.colab import auth
auth.authenticate_user()

import os
PROJECT_ID = input("Enter your GCP Project ID: ").strip()
REGION = "us-central1"  # keep consistent; change if instructed
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["REGION"] = REGION # Export the REGION environment variable
print("Project:", PROJECT_ID, "| Region:", REGION)

# Set active project for gcloud/BigQuery CLI
!gcloud config set project $GOOGLE_CLOUD_PROJECT
!gcloud config get-value project
# Done: Auth + Project/Region set

Enter your GCP Project ID: mgmt-467-94721
Project: mgmt-467-94721 | Region: us-central1
Updated property [core/project].
mgmt-467-94721


### Verification Prompt
Generate a short cell that prints the active project using `gcloud config get-value project` and echoes the `REGION` you set.


In [ ]:
# Verification step: Check active project and region
!gcloud config get-value project
import os
print("Project:", PROJECT_ID, "| Region:", REGION)

mgmt-467-94721
Project: mgmt-467-94721 | Region: us-central1


**Reflection:** Why do we set `PROJECT_ID` and `REGION` at the top? What can go wrong if we don’t?

## 1) Kaggle API — What & Why
Use Kaggle CLI for reproducible downloads. Store `kaggle.json` at `~/.kaggle/kaggle.json` with `0600` permissions to protect secrets.

### Build Prompt
Generate a **single Colab code cell** that:
- Prompts me to upload `kaggle.json`,
- Saves to `~/.kaggle/kaggle.json` with `0600` permissions,
- Prints `kaggle --version`.
Add comments about security and reproducibility.


In [ ]:
# EXAMPLE (from LLM) — Kaggle setup (commented)
from google.colab import files
print("Upload your kaggle.json (Kaggle > Account > Create New API Token)")
uploaded = files.upload()

import os
os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'wb') as f:
    f.write(uploaded[list(uploaded.keys())[0]])
os.chmod('/root/.kaggle/kaggle.json', 0o600)  # owner-only

!kaggle --version

Upload your kaggle.json (Kaggle > Account > Create New API Token)


Saving kaggle.json to kaggle (1).json
Kaggle API 1.7.4.5


### Verification Prompt
Generate a one-liner that runs `kaggle --help | head -n 20` to show the CLI is ready.


**Reflection:** Why require strict `0600` permissions on API tokens? What risks are we avoiding?

## 2) Download & unzip dataset — What & Why
Keep raw files under `/content/data/raw` for predictable paths and auditing.
**Dataset:** `sayeeduddin/netflix-2025user-behavior-dataset-210k-records`

### Build Prompt
Generate a **Colab code cell** that:
- Creates `/content/data/raw`,
- Downloads the dataset to `/content/data` with Kaggle CLI,
- Unzips into `/content/data/raw` (overwrite OK),
- Lists all CSVs with sizes in a neat table.
Include comments describing each step.


In [ ]:
# Create the directory to store raw data
!mkdir -p /content/data/raw

# Download the dataset using Kaggle CLI to /content/data
# The -d flag specifies the dataset, and -p specifies the download path
!kaggle datasets download -d sayeeduddin/netflix-2025user-behavior-dataset-210k-records -p /content/data

# Unzip the downloaded dataset into the raw data directory
# -o flag overwrites files if they exist
!unzip -o /content/data/*.zip -d /content/data/raw

# List all CSV files in the raw data directory with their sizes in a neat table
!ls -lh /content/data/raw/*.csv

Dataset URL: https://www.kaggle.com/datasets/sayeeduddin/netflix-2025user-behavior-dataset-210k-records
License(s): CC0-1.0
  0% 0.00/4.02M [00:00<?, ?B/s]
100% 4.02M/4.02M [00:00<00:00, 682MB/s]
Archive:  /content/data/netflix-2025user-behavior-dataset-210k-records.zip
  inflating: /content/data/raw/README.md  
  inflating: /content/data/raw/movies.csv  
  inflating: /content/data/raw/recommendation_logs.csv  
  inflating: /content/data/raw/reviews.csv  
  inflating: /content/data/raw/search_logs.csv  
  inflating: /content/data/raw/users.csv  
  inflating: /content/data/raw/watch_history.csv  
-rw-r--r-- 1 root root 114K Aug  2 19:36 /content/data/raw/movies.csv
-rw-r--r-- 1 root root 4.5M Aug  2 19:36 /content/data/raw/recommendation_logs.csv
-rw-r--r-- 1 root root 1.8M Aug  2 19:36 /content/data/raw/reviews.csv
-rw-r--r-- 1 root root 2.2M Aug  2 19:36 /content/data/raw/search_logs.csv
-rw-r--r-- 1 root root 1.6M Aug  2 19:36 /content/data/raw/users.csv
-rw-r--r-- 1 root root 8.9M A

### Verification Prompt
Generate a snippet that asserts there are exactly **six** CSV files and prints their names.


**Reflection:** Why is keeping a clean file inventory (names, sizes) useful downstream?

## 3) Create GCS bucket & upload — What & Why
Stage in GCS → consistent, versionable source for BigQuery loads. Bucket names must be **globally unique**.

### Build Prompt
Generate a **Colab code cell** that:
- Creates a unique bucket in `${REGION}` (random suffix),
- Saves name to `BUCKET_NAME` env var,
- Uploads all CSVs to `gs://$BUCKET_NAME/netflix/`,
- Prints the bucket name and explains staging benefits.


In [ ]:
# Create a unique bucket in the specified region
import uuid
import os

bucket_name = f"mgmt467-netflix-{uuid.uuid4().hex[:8]}"
os.environ["BUCKET_NAME"] = bucket_name

# Correctly access the REGION environment variable
REGION = os.environ.get('REGION')
print(f"Creating bucket: gs://{bucket_name} in region: {REGION}")
!gcloud storage buckets create gs://$BUCKET_NAME --location=$REGION

# Upload all CSVs to the bucket
print(f"\nUploading files to gs://{bucket_name}/netflix/")
!gcloud storage cp /content/data/raw/* gs://$BUCKET_NAME/netflix/

# Print the bucket name and explain staging benefits
print(f"\nSuccessfully created bucket: {bucket_name} and uploaded files.")
print("\nBenefits of staging data in GCS:")
print("- **Consistent Source:** GCS provides a stable and versionable location for your data.")
print("- **Scalability:** GCS is highly scalable, handling large datasets easily.")
print("- **Integration:** GCS integrates seamlessly with other Google Cloud services like BigQuery.")
print("- **Cost-Effective:** GCS can be a cost-effective storage solution.")

# Verify contents (optional but recommended)
print(f"\nVerifying contents of gs://{bucket_name}/netflix/:")
!gcloud storage ls gs://$BUCKET_NAME/netflix/

Creating bucket: gs://mgmt467-netflix-b7812b55 in region: us-central1
Creating gs://mgmt467-netflix-b7812b55/...

Uploading files to gs://mgmt467-netflix-b7812b55/netflix/
Copying file:///content/data/raw/movies.csv to gs://mgmt467-netflix-b7812b55/netflix/movies.csv
Copying file:///content/data/raw/README.md to gs://mgmt467-netflix-b7812b55/netflix/README.md
Copying file:///content/data/raw/recommendation_logs.csv to gs://mgmt467-netflix-b7812b55/netflix/recommendation_logs.csv
Copying file:///content/data/raw/reviews.csv to gs://mgmt467-netflix-b7812b55/netflix/reviews.csv
Copying file:///content/data/raw/search_logs.csv to gs://mgmt467-netflix-b7812b55/netflix/search_logs.csv
Copying file:///content/data/raw/users.csv to gs://mgmt467-netflix-b7812b55/netflix/users.csv
Copying file:///content/data/raw/watch_history.csv to gs://mgmt467-netflix-b7812b55/netflix/watch_history.csv

Average throughput: 30.9MiB/s

Successfully created bucket: mgmt467-netflix-b7812b55 and uploaded files.

B

### Verification Prompt
Generate a snippet that lists the `netflix/` prefix and shows object sizes.


**Reflection:** Name two benefits of staging in GCS vs loading directly from local Colab.

## 4) BigQuery dataset & loads — What & Why
Create dataset `netflix` and load six CSVs with **autodetect** for speed (we’ll enforce schemas later).

### Build Prompt (two cells)
**Cell A:** Create (idempotently) dataset `netflix` in US multi-region; if it exists, print a friendly message.  
**Cell B:** Load tables from `gs://$BUCKET_NAME/netflix/`:
`users, movies, watch_history, recommendation_logs, search_logs, reviews`
with `--skip_leading_rows=1 --autodetect --source_format=CSV`.
Finish with row-count queries for each table.


In [ ]:
# EXAMPLE (from LLM) — BigQuery dataset (commented)
DATASET="netflix"
# Attempt to create; ignore if exists
!bq --location=US mk -d --description "MGMT467 Netflix dataset" $DATASET || echo "Dataset may already exist."

Dataset 'mgmt-467-94721:netflix' successfully created.


In [ ]:
# EXAMPLE (from LLM) — Load tables (commented)
tables = {
  "users": "users.csv",
  "movies": "movies.csv",
  "watch_history": "watch_history.csv",
  "recommendation_logs": "recommendation_logs.csv",
  "search_logs": "search_logs.csv",
  "reviews": "reviews.csv",
}
import os
DATASET="netflix" # Make sure dataset is defined
for tbl, fname in tables.items():
  src = f"gs://{os.environ['BUCKET_NAME']}/netflix/{fname}"
  print("Loading", tbl, "from", src)
  # Corrected bq load command
  !bq load --skip_leading_rows=1 --autodetect --source_format=CSV {DATASET}.{tbl} {src}

# Row counts
for tbl in tables.keys():
  # Corrected bq query command using f-string
  print(f"Counting rows in {tbl}")
  !bq query --nouse_legacy_sql "SELECT '{tbl}' AS table_name, COUNT(*) AS n FROM \`{os.environ['GOOGLE_CLOUD_PROJECT']}.netflix.{tbl}\`"

Loading users from gs://mgmt467-netflix-b7812b55/netflix/users.csv
Waiting on bqjob_r14b1986a573696ae_0000019a12817c39_1 ... (4s) Current status: DONE   
Loading movies from gs://mgmt467-netflix-b7812b55/netflix/movies.csv
Waiting on bqjob_r359e11d1a4fe3a92_0000019a1281a1c0_1 ... (1s) Current status: DONE   
Loading watch_history from gs://mgmt467-netflix-b7812b55/netflix/watch_history.csv
Waiting on bqjob_r20c27efa2cb0da64_0000019a1281b99d_1 ... (2s) Current status: DONE   
Loading recommendation_logs from gs://mgmt467-netflix-b7812b55/netflix/recommendation_logs.csv
Waiting on bqjob_r16dfee57f38005d3_0000019a1281d5d9_1 ... (2s) Current status: DONE   
Loading search_logs from gs://mgmt467-netflix-b7812b55/netflix/search_logs.csv
Waiting on bqjob_r46e66171e45e4f37_0000019a1281f348_1 ... (1s) Current status: DONE   
Loading reviews from gs://mgmt467-netflix-b7812b55/netflix/reviews.csv
Waiting on bqjob_r2544c8474ef73b66_0000019a12820a5d_1 ... (1s) Current status: DONE   
Counting rows 

### Verification Prompt
Generate a single query that returns `table_name, row_count` for all six tables in `${GOOGLE_CLOUD_PROJECT}.netflix`.


**Reflection:** When is `autodetect` acceptable? When should you enforce explicit schemas and why?

## 5) Data Quality (DQ) — Concepts we care about
- **Missingness** (MCAR/MAR/MNAR). Impute vs drop. Add `is_missing_*` indicators.
- **Duplicates** (exact vs near). Double-counted engagement corrupts labels & KPIs.
- **Outliers** (IQR). Winsorize/cap vs robust models. Always **flag** and explain.
- **Reproducibility**. Prefer `CREATE OR REPLACE` and deterministic keys.


### 5.1 Missingness (users) — What & Why
Measure % missing and check if missingness depends on another variable (MAR) → potential bias & instability.

### Build Prompt
Generate **two BigQuery SQL cells**:
1) Total rows and % missing in `region`, `plan_tier`, `age_band` from `users`.
2) `% plan_tier missing by region` ordered descending. Add comments on MAR.


In [ ]:
# EXAMPLE (from LLM) — Missingness profile (commented)
import os
from google.cloud import bigquery

project_id = os.environ['GOOGLE_CLOUD_PROJECT']

client = bigquery.Client(project=project_id)

query = f"""
-- Users: % missing per column
WITH base AS (
  SELECT COUNT(*) n,
         COUNTIF(country IS NULL) miss_country,
         COUNTIF(subscription_plan IS NULL) miss_plan,
         COUNTIF(age IS NULL) miss_age
  FROM `{project_id}.netflix.users`
)
SELECT n,
       ROUND(100*miss_country/n,2) AS pct_missing_country,
       ROUND(100*miss_plan/n,2)   AS pct_missing_subscription_plan,
       ROUND(100*miss_age/n,2)    AS pct_missing_age
FROM base;
"""

query_job = client.query(query)
results = query_job.result()

# Print the results
for row in results:
    print(row)

Row((10300, 0.0, 0.0, 11.93), {'n': 0, 'pct_missing_country': 1, 'pct_missing_subscription_plan': 2, 'pct_missing_age': 3})


### Save & submit checklist:

- [ ] Save this notebook to the team Drive.
- [ ] Export a `.sql` file with your DQ queries and save to repo.
- [ ] Push notebook + SQL to the **team GitHub** with a descriptive commit.
- [ ] Add a README with your `PROJECT_ID`, `REGION`, bucket, dataset, and today’s row counts.

In [ ]:
# Verification: Single compact summary query for anomaly flags
import os
from google.cloud import bigquery

project_id = os.environ['GOOGLE_CLOUD_PROJECT']
client = bigquery.Client(project=project_id)

query = f"""
SELECT
  'flag_binge' AS flag_name,
  ROUND(100*COUNTIF(watch_duration_minutes_capped > 8*60)/COUNT(*),2) AS pct_of_rows
FROM `{project_id}.netflix.watch_history_robust`
UNION ALL
SELECT
  'flag_age_extreme' AS flag_name,
  ROUND(100*COUNTIF(age < 10 OR age > 100)/COUNT(*),2) AS pct_of_rows
FROM `{project_id}.netflix.users`
UNION ALL
SELECT
  'flag_duration_anomaly' AS flag_name,
  ROUND(100*COUNTIF(duration_minutes < 15 OR duration_minutes > 480)/COUNT(*),2) AS pct_of_rows
FROM `{project_id}.netflix.movies`;
"""

query_job = client.query(query)
results = query_job.result()

# Print the results
for row in results:
    print(row)

BadRequest: 400 Unrecognized name: watch_duration_minutes_capped; Did you mean watch_duration_minutes? at [4:21]; reason: invalidQuery, location: query, message: Unrecognized name: watch_duration_minutes_capped; Did you mean watch_duration_minutes? at [4:21]

Location: US
Job ID: 5e4609c2-9096-47c4-8b00-1c5af90391e6


In [ ]:
# In movies, compute and summarize flag_duration_anomaly where duration_minutes < 15 or > 480.
import os
from google.cloud import bigquery

project_id = os.environ['GOOGLE_CLOUD_PROJECT']
client = bigquery.Client(project=project_id)

query = f"""
SELECT
  COUNTIF(duration_minutes < 15) AS titles_under_15m,
  COUNTIF(duration_minutes > 480) AS titles_over_8h,
  COUNT(*) AS total,
  ROUND(100*COUNTIF(duration_minutes < 15 OR duration_minutes > 480)/COUNT(*),2) AS pct_duration_anomaly
FROM `{project_id}.netflix.movies`;
"""

query_job = client.query(query)
results = query_job.result()

# Print the results
for row in results:
    print(row)

Row((24, 22, 2080, 2.21), {'titles_under_15m': 0, 'titles_over_8h': 1, 'total': 2, 'pct_duration_anomaly': 3})


In [ ]:
import os
project_id = os.environ['GOOGLE_CLOUD_PROJECT']
!bq head --schema=true {project_id}.netflix.movies

FATAL Flags parsing error: Unknown command line flag 'schema'
Run 'bq.py help' to get help


In [ ]:
# In users, compute and summarize flag_age_extreme if age is <10 or >100.
import os
from google.cloud import bigquery

project_id = os.environ['GOOGLE_CLOUD_PROJECT']
client = bigquery.Client(project=project_id)

query = f"""
SELECT
  COUNTIF(age < 10 OR age > 100) AS extreme_age_rows,
  COUNT(*) AS total,
  ROUND(100*COUNTIF(age < 10 OR age > 100)/COUNT(*),2) AS pct
FROM `{project_id}.netflix.users`;
"""

query_job = client.query(query)
results = query_job.result()

# Print the results
for row in results:
    print(row)

Row((358, 20600, 1.74), {'extreme_age_rows': 0, 'total': 1, 'pct': 2})


In [ ]:
# In watch_history_robust, compute and summarize flag_binge for sessions > 8 hours.
import os
from google.cloud import bigquery

project_id = os.environ['GOOGLE_CLOUD_PROJECT']
client = bigquery.Client(project=project_id)

query = f"""
SELECT
  COUNTIF(watch_duration_minutes_capped > 8*60) AS sessions_over_8h,
  COUNT(*) AS total,
  ROUND(100*COUNTIF(watch_duration_minutes_capped > 8*60)/COUNT(*),2) AS pct
FROM `{project_id}.netflix.watch_history_robust`;
"""

query_job = client.query(query)
results = query_job.result()

# Print the results
for row in results:
    print(row)

Row((0, 100000, 0.0), {'sessions_over_8h': 0, 'total': 1, 'pct': 2})


In [ ]:
# Verification: Show min/median/max before vs after capping
import os
from google.cloud import bigquery

project_id = os.environ['GOOGLE_CLOUD_PROJECT']
client = bigquery.Client(project=project_id)

query = f"""
WITH before AS (
  SELECT
    'before' AS which,
    MIN(watch_duration_minutes) AS min_val,
    APPROX_QUANTILES(watch_duration_minutes, 2)[OFFSET(1)] AS median_val,
    MAX(watch_duration_minutes) AS max_val
  FROM `{project_id}.netflix.watch_history_dedup`
),
after AS (
  SELECT
    'after' AS which,
    MIN(watch_duration_minutes_capped) AS min_val,
    APPROX_QUANTILES(watch_duration_minutes_capped, 2)[OFFSET(1)] AS median_val,
    MAX(watch_duration_minutes_capped) AS max_val
  FROM `{project_id}.netflix.watch_history_robust`
)
SELECT * FROM before UNION ALL SELECT * FROM after;
"""

query_job = client.query(query)
results = query_job.result()

# Print the results
for row in results:
    print(row)

Row(('after', 4.4, 51.4, 203.6), {'which': 0, 'min_val': 1, 'median_val': 2, 'max_val': 3})
Row(('before', 0.2, 51.1, 799.3), {'which': 0, 'min_val': 1, 'median_val': 2, 'max_val': 3})


In [ ]:
# Create watch_history_robust with watch_duration_minutes_capped at P01/P99; return quantile summaries before/after.
import os
from google.cloud import bigquery

project_id = os.environ['GOOGLE_CLOUD_PROJECT']
client = bigquery.Client(project=project_id)

query = f"""
CREATE OR REPLACE TABLE `{project_id}.netflix.watch_history_robust` AS
WITH q AS (
  SELECT
    APPROX_QUANTILES(watch_duration_minutes, 100)[OFFSET(1)]  AS p01,
    APPROX_QUANTILES(watch_duration_minutes, 100)[OFFSET(98)] AS p99
  FROM `{project_id}.netflix.watch_history_dedup`
)
SELECT
  h.*,
  GREATEST(q.p01, LEAST(q.p99, h.watch_duration_minutes)) AS watch_duration_minutes_capped
FROM `{project_id}.netflix.watch_history_dedup` h, q;

-- Quantiles before vs after
WITH before AS (
  SELECT 'before' AS which, APPROX_QUANTILES(watch_duration_minutes, 5) AS q
  FROM `{project_id}.netflix.watch_history_dedup`
),
after AS (
  SELECT 'after' AS which, APPROX_QUANTILES(watch_duration_minutes_capped, 5) AS q
  FROM `{project_id}.netflix.watch_history_robust`
)
SELECT * FROM before UNION ALL SELECT * FROM after;
"""

query_job = client.query(query)
results = query_job.result()

# Print the results
for row in results:
    print(row)

Row(('after', [4.4, 24.6, 41.5, 61.5, 92.0, 203.6]), {'which': 0, 'q': 1})
Row(('before', [0.2, 24.8, 41.7, 61.2, 91.7, 799.3]), {'which': 0, 'q': 1})


In [ ]:
# Compute IQR bounds for watch_duration_minutes on watch_history_dedup and report % outliers
import os
from google.cloud import bigquery

project_id = os.environ['GOOGLE_CLOUD_PROJECT']
client = bigquery.Client(project=project_id)

query = f"""
WITH dist AS (
  SELECT
    APPROX_QUANTILES(watch_duration_minutes, 4)[OFFSET(1)] AS q1,
    APPROX_QUANTILES(watch_duration_minutes, 4)[OFFSET(3)] AS q3
  FROM `{project_id}.netflix.watch_history_dedup`
),
bounds AS (
  SELECT q1, q3, (q3-q1) AS iqr,
         q1 - 1.5*(q3-q1) AS lo,
         q3 + 1.5*(q3-q1) AS hi
  FROM dist
)
SELECT
  COUNTIF(h.watch_duration_minutes < b.lo OR h.watch_duration_minutes > b.hi) AS outliers,
  COUNT(*) AS total,
  ROUND(100*COUNTIF(h.watch_duration_minutes < b.lo OR h.watch_duration_minutes > b.hi)/COUNT(*),2) AS pct_outliers
FROM `{project_id}.netflix.watch_history_dedup` h
CROSS JOIN bounds b;
"""

query_job = client.query(query)
results = query_job.result()

# Print the results
for row in results:
    print(row)

Row((3552, 100000, 3.55), {'outliers': 0, 'total': 1, 'pct_outliers': 2})


In [ ]:
# Verification: Before/after count query comparing raw vs watch_history_dedup
import os
from google.cloud import bigquery

project_id = os.environ['GOOGLE_CLOUD_PROJECT']
client = bigquery.Client(project=project_id)

query = f"""
SELECT 'raw' AS table_name, COUNT(*) AS row_count FROM `{project_id}.netflix.watch_history`
UNION ALL
SELECT 'deduplicated' AS table_name, COUNT(*) AS row_count FROM `{project_id}.netflix.watch_history_dedup`;
"""

query_job = client.query(query)
results = query_job.result()

# Print the results
for row in results:
    print(row)

Row(('deduplicated', 100000), {'table_name': 0, 'row_count': 1})
Row(('raw', 210000), {'table_name': 0, 'row_count': 1})


In [ ]:
import os
from google.cloud import bigquery

project_id = os.environ['GOOGLE_CLOUD_PROJECT']
client = bigquery.Client(project=project_id)

query = f"SELECT h.* FROM `{project_id}.netflix.watch_history` h LIMIT 1"
query_job = client.query(query)
results = query_job.result()

for row in results:
    print(row.keys())

dict_keys(['session_id', 'user_id', 'movie_id', 'watch_date', 'device_type', 'watch_duration_minutes', 'progress_percentage', 'action', 'quality', 'location_country', 'is_download', 'user_rating'])


In [ ]:
import os
project_id = os.environ['GOOGLE_CLOUD_PROJECT']
!bq head --schema=true {project_id}.netflix.watch_history

FATAL Flags parsing error: Unknown command line flag 'schema'
Run 'bq.py help' to get help


In [ ]:
# Detect duplicate groups on (user_id, movie_id, watch_date, device_type) with counts (top 20)
import os
from google.cloud import bigquery

project_id = os.environ['GOOGLE_CLOUD_PROJECT']
client = bigquery.Client(project=project_id)

query = f"""
SELECT user_id, movie_id, watch_date, device_type, COUNT(*) AS dup_count
FROM `{project_id}.netflix.watch_history`
GROUP BY user_id, movie_id, watch_date, device_type
HAVING dup_count > 1
ORDER BY dup_count DESC
LIMIT 20;
"""

query_job = client.query(query)
results = query_job.result()

# Print the results
for row in results:
    print(row)

Row(('user_03310', 'movie_0640', datetime.date(2024, 9, 8), 'Smart TV', 8), {'user_id': 0, 'movie_id': 1, 'watch_date': 2, 'device_type': 3, 'dup_count': 4})
Row(('user_00391', 'movie_0893', datetime.date(2024, 8, 26), 'Laptop', 8), {'user_id': 0, 'movie_id': 1, 'watch_date': 2, 'device_type': 3, 'dup_count': 4})
Row(('user_06417', 'movie_0590', datetime.date(2024, 1, 15), 'Laptop', 6), {'user_id': 0, 'movie_id': 1, 'watch_date': 2, 'device_type': 3, 'dup_count': 4})
Row(('user_07594', 'movie_0133', datetime.date(2025, 3, 24), 'Laptop', 6), {'user_id': 0, 'movie_id': 1, 'watch_date': 2, 'device_type': 3, 'dup_count': 4})
Row(('user_00928', 'movie_0913', datetime.date(2024, 1, 18), 'Laptop', 6), {'user_id': 0, 'movie_id': 1, 'watch_date': 2, 'device_type': 3, 'dup_count': 4})
Row(('user_03021', 'movie_0602', datetime.date(2025, 2, 23), 'Laptop', 6), {'user_id': 0, 'movie_id': 1, 'watch_date': 2, 'device_type': 3, 'dup_count': 4})
Row(('user_00249', 'movie_0203', datetime.date(2024, 8, 3

In [ ]:
import os
from google.cloud import bigquery

project_id = os.environ['GOOGLE_CLOUD_PROJECT']
client = bigquery.Client(project=project_id)

query = f"""
-- Verification: Print missingness percentages
WITH base AS (
  SELECT COUNT(*) n,
         COUNTIF(country IS NULL) miss_country,
         COUNTIF(subscription_plan IS NULL) miss_plan,
         COUNTIF(age IS NULL) miss_age
  FROM `{project_id}.netflix.users`
)
SELECT ROUND(100*miss_country/n,2) AS pct_missing_country,
       ROUND(100*miss_plan/n,2)   AS pct_missing_subscription_plan,
       ROUND(100*miss_age/n,2)    AS pct_missing_age
FROM base;
"""

query_job = client.query(query)
results = query_job.result()

# Print the results
for row in results:
    print(row)

Row((0.0, 0.0, 11.93), {'pct_missing_country': 0, 'pct_missing_subscription_plan': 1, 'pct_missing_age': 2})


In [ ]:
import os
from google.cloud import bigquery

project_id = os.environ['GOOGLE_CLOUD_PROJECT']
client = bigquery.Client(project=project_id)

query = f"SELECT * FROM `{project_id}.netflix.users` LIMIT 1"
query_job = client.query(query)
results = query_job.result()

for row in results:
    print(row.keys())

dict_keys(['user_id', 'email', 'first_name', 'last_name', 'age', 'gender', 'country', 'state_province', 'city', 'subscription_plan', 'subscription_start_date', 'is_active', 'monthly_spend', 'primary_device', 'household_size', 'created_at'])


In [ ]:
import os
project_id = os.environ['GOOGLE_CLOUD_PROJECT']
!bq head --schema=true {project_id}.netflix.users

FATAL Flags parsing error: Unknown command line flag 'schema'
Run 'bq.py help' to get help


In [ ]:
%load_ext bigquery_magics

The bigquery_magics extension is already loaded. To reload it, use:
  %reload_ext bigquery_magics


In [ ]:
# # EXAMPLE (from LLM) — MAR by region (commented)
import os
from google.cloud import bigquery

project_id = os.environ['GOOGLE_CLOUD_PROJECT']
client = bigquery.Client(project=project_id)

query = f"""
SELECT country,
       COUNT(*) AS n,
       ROUND(100*COUNTIF(subscription_plan IS NULL)/COUNT(*),2) AS pct_missing_subscription_plan
FROM `{project_id}.netflix.users`
GROUP BY country
ORDER BY pct_missing_subscription_plan DESC;
"""

query_job = client.query(query)
results = query_job.result()

# Print the results
for row in results:
    print(row)

Row(('Canada', 6192, 0.0), {'country': 0, 'n': 1, 'pct_missing_subscription_plan': 2})
Row(('USA', 14408, 0.0), {'country': 0, 'n': 1, 'pct_missing_subscription_plan': 2})


### Verification Prompt
Generate a query that prints the three missingness percentages from (1), rounded to two decimals.


**Reflection:** Which columns are most missing? Hypothesize MCAR/MAR/MNAR and why.

### 5.2 Duplicates (watch_history) — What & Why
Find exact duplicate interaction records and keep **one best** per group (deterministic policy).

### Build Prompt
Generate **two BigQuery SQL cells**:
1) Report duplicate groups on `(user_id, movie_id, event_ts, device_type)` with counts (top 20).
2) Create table `watch_history_dedup` that keeps one row per group (prefer higher `progress_ratio`, then `minutes_watched`). Add comments.


In [ ]:
# # EXAMPLE (from LLM) — Detect duplicate groups (commented)
import os
from google.cloud import bigquery

project_id = os.environ['GOOGLE_CLOUD_PROJECT']
client = bigquery.Client(project=project_id)

query = f"""
SELECT user_id, movie_id, watch_date, device_type, COUNT(*) AS dup_count
FROM `{project_id}.netflix.watch_history`
GROUP BY user_id, movie_id, watch_date, device_type
HAVING dup_count > 1
ORDER BY dup_count DESC
LIMIT 20;
"""

query_job = client.query(query)
results = query_job.result()

# Print the results
for row in results:
    print(row)

Row(('user_03310', 'movie_0640', datetime.date(2024, 9, 8), 'Smart TV', 8), {'user_id': 0, 'movie_id': 1, 'watch_date': 2, 'device_type': 3, 'dup_count': 4})
Row(('user_00391', 'movie_0893', datetime.date(2024, 8, 26), 'Laptop', 8), {'user_id': 0, 'movie_id': 1, 'watch_date': 2, 'device_type': 3, 'dup_count': 4})
Row(('user_06417', 'movie_0590', datetime.date(2024, 1, 15), 'Laptop', 6), {'user_id': 0, 'movie_id': 1, 'watch_date': 2, 'device_type': 3, 'dup_count': 4})
Row(('user_07594', 'movie_0133', datetime.date(2025, 3, 24), 'Laptop', 6), {'user_id': 0, 'movie_id': 1, 'watch_date': 2, 'device_type': 3, 'dup_count': 4})
Row(('user_00928', 'movie_0913', datetime.date(2024, 1, 18), 'Laptop', 6), {'user_id': 0, 'movie_id': 1, 'watch_date': 2, 'device_type': 3, 'dup_count': 4})
Row(('user_03021', 'movie_0602', datetime.date(2025, 2, 23), 'Laptop', 6), {'user_id': 0, 'movie_id': 1, 'watch_date': 2, 'device_type': 3, 'dup_count': 4})
Row(('user_00249', 'movie_0203', datetime.date(2024, 8, 3

In [3]:
# # EXAMPLE (from LLM) — Keep-one policy (commented)
import os
from google.cloud import bigquery

project_id = os.environ['mgmt-467-94721']
client = bigquery.Client(project=project_id)

query = f"""
CREATE OR REPLACE TABLE `{project_id}.netflix.watch_history_dedup` AS
SELECT * EXCEPT(rk) FROM (
  SELECT h.*,
         ROW_NUMBER() OVER (
           PARTITION BY user_id, movie_id, watch_date, device_type
           ORDER BY progress_percentage DESC, watch_duration_minutes DESC
         ) AS rk
  FROM `{project_id}.netflix.watch_history` h
)
WHERE rk = 1;
"""

query_job = client.query(query)
results = query_job.result()

print("Deduplicated table watch_history_dedup created successfully.")

KeyError: 'mgmt-467-94721'

### Verification Prompt
Generate a before/after count query comparing raw vs `watch_history_dedup`.


**Reflection:** Why do duplicates arise (natural vs system-generated)? How do they corrupt labels and KPIs?

### 5.3 Outliers (minutes_watched) — What & Why
Estimate extreme values via IQR; report % outliers; **winsorize** to P01/P99 for robustness while also **flagging** extremes.

### Build Prompt
Generate **two BigQuery SQL cells**:
1) Compute IQR bounds for `minutes_watched` on `watch_history_dedup` and report % outliers.
2) Create `watch_history_robust` with `minutes_watched_capped` capped at P01/P99; return quantile summaries before/after.


In [ ]:
# # EXAMPLE (from LLM) — IQR outlier rate (commented)
import os
from google.cloud import bigquery

project_id = os.environ['GOOGLE_CLOUD_PROJECT']
client = bigquery.Client(project=project_id)

query = f"""
WITH dist AS (
  SELECT
    APPROX_QUANTILES(watch_duration_minutes, 4)[OFFSET(1)] AS q1,
    APPROX_QUANTILES(watch_duration_minutes, 4)[OFFSET(3)] AS q3
  FROM `{project_id}.netflix.watch_history_dedup`
),
bounds AS (
  SELECT q1, q3, (q3-q1) AS iqr,
         q1 - 1.5*(q3-q1) AS lo,
         q3 + 1.5*(q3-q1) AS hi
  FROM dist
)
SELECT
  COUNTIF(h.watch_duration_minutes < b.lo OR h.watch_duration_minutes > b.hi) AS outliers,
  COUNT(*) AS total,
  ROUND(100*COUNTIF(h.watch_duration_minutes < b.lo OR h.watch_duration_minutes > b.hi)/COUNT(*),2) AS pct_outliers
FROM `{project_id}.netflix.watch_history_dedup` h
CROSS JOIN bounds b;
"""

query_job = client.query(query)
results = query_job.result()

# Print the results
for row in results:
    print(row)

NotFound: 404 Not found: Table mgmt-467-94721:netflix.watch_history_dedup was not found in location US; reason: notFound, message: Not found: Table mgmt-467-94721:netflix.watch_history_dedup was not found in location US

Location: US
Job ID: 011d8e5d-3a4e-4d30-85c1-bfc29d2a8424


In [ ]:
# # EXAMPLE (from LLM) — Winsorize + quantiles (commented)
import os
from google.cloud import bigquery

project_id = os.environ['GOOGLE_CLOUD_PROJECT']
client = bigquery.Client(project=project_id)

query = f"""
CREATE OR REPLACE TABLE `{project_id}.netflix.watch_history_robust` AS
WITH q AS (
  SELECT
    APPROX_QUANTILES(watch_duration_minutes, 100)[OFFSET(1)]  AS p01,
    APPROX_QUANTILES(watch_duration_minutes, 100)[OFFSET(98)] AS p99
  FROM `{project_id}.netflix.watch_history_dedup`
)
SELECT
  h.*,
  GREATEST(q.p01, LEAST(q.p99, h.watch_duration_minutes)) AS watch_duration_minutes_capped
FROM `{project_id}.netflix.watch_history_dedup` h, q;

-- Quantiles before vs after
WITH before AS (
  SELECT 'before' AS which, APPROX_QUANTILES(watch_duration_minutes, 5) AS q
  FROM `{project_id}.netflix.watch_history_dedup`
),
after AS (
  SELECT 'after' AS which, APPROX_QUANTILES(watch_duration_minutes_capped, 5) AS q
  FROM `{project_id}.netflix.watch_history_robust`
)
SELECT * FROM before UNION ALL SELECT * FROM after;
"""

query_job = client.query(query)
results = query_job.result()

# Print the results
for row in results:
    print(row)

Row(('after', [4.4, 24.6, 41.5, 61.5, 92.0, 203.6]), {'which': 0, 'q': 1})
Row(('before', [0.2, 24.8, 41.8, 61.3, 91.7, 799.3]), {'which': 0, 'q': 1})


### Verification Prompt
Generate a query that shows min/median/max before vs after capping.


**Reflection:** When might capping be harmful? Name a model type less sensitive to outliers and why.

### 5.4 Business anomaly flags — What & Why
Human-readable flags help both product decisioning and ML features (e.g., binge behavior).

### Build Prompt
Generate **three BigQuery SQL cells** (adjust if columns differ):
1) In `watch_history_robust`, compute and summarize `flag_binge` for sessions > 8 hours.
2) In `users`, compute and summarize `flag_age_extreme` if age can be parsed from `age_band` (<10 or >100).
3) In `movies`, compute and summarize `flag_duration_anomaly` where `duration_min` < 15 or > 480 (if exists).
Each cell should output count and percentage and include 1–2 comments.


In [ ]:
# # EXAMPLE (from LLM) — flag_binge (commented)
import os
from google.cloud import bigquery

project_id = os.environ['GOOGLE_CLOUD_PROJECT']
client = bigquery.Client(project=project_id)

query = f"""
SELECT
  COUNTIF(watch_duration_minutes_capped > 8*60) AS sessions_over_8h,
  COUNT(*) AS total,
  ROUND(100*COUNTIF(watch_duration_minutes_capped > 8*60)/COUNT(*),2) AS pct
FROM `{project_id}.netflix.watch_history_robust`;
"""

query_job = client.query(query)
results = query_job.result()

# Print the results
for row in results:
    print(row)

Row((0, 100000, 0.0), {'sessions_over_8h': 0, 'total': 1, 'pct': 2})


In [ ]:
# # EXAMPLE (from LLM) — flag_age_extreme (commented)
import os
from google.cloud import bigquery

project_id = os.environ['GOOGLE_CLOUD_PROJECT']
client = bigquery.Client(project=project_id)

query = f"""
SELECT
  COUNTIF(age < 10 OR age > 100) AS extreme_age_rows,
  COUNT(*) AS total,
  ROUND(100*COUNTIF(age < 10 OR age > 100)/COUNT(*),2) AS pct
FROM `{project_id}.netflix.users`;
"""

query_job = client.query(query)
results = query_job.result()

# Print the results
for row in results:
    print(row)

Row((358, 20600, 1.74), {'extreme_age_rows': 0, 'total': 1, 'pct': 2})


In [ ]:
# # EXAMPLE (from LLM) — flag_duration_anomaly (commented)
import os
from google.cloud import bigquery

project_id = os.environ['GOOGLE_CLOUD_PROJECT']
client = bigquery.Client(project=project_id)

query = f"""
SELECT
  COUNTIF(duration_minutes < 15) AS titles_under_15m,
  COUNTIF(duration_minutes > 480) AS titles_over_8h,
  COUNT(*) AS total,
  ROUND(100*COUNTIF(duration_minutes < 15 OR duration_minutes > 480)/COUNT(*),2) AS pct_duration_anomaly
FROM `{project_id}.netflix.movies`;
"""

query_job = client.query(query)
results = query_job.result()

# Print the results
for row in results:
    print(row)

Row((24, 22, 2080, 2.21), {'titles_under_15m': 0, 'titles_over_8h': 1, 'total': 2, 'pct_duration_anomaly': 3})


### Verification Prompt
Generate a single compact summary query that returns two columns per flag: `flag_name, pct_of_rows`.


**Reflection:** Which anomaly flag is most common? Which would you keep as a feature and why?

## 6) Save & submit — What & Why
Reproducibility: save artifacts and document decisions so others can rerun and audit.

### Build Prompt
Generate a checklist (Markdown) students can paste at the end:
- Save this notebook to the team Drive.
- Export a `.sql` file with your DQ queries and save to repo.
- Push notebook + SQL to the **team GitHub** with a descriptive commit.
- Add a README with your `PROJECT_ID`, `REGION`, bucket, dataset, and today’s row counts.


## Grading rubric (quick)
- Profiling completeness (30)  
- Cleaning policy correctness & reproducibility (40)  
- Reflection/insight (20)  
- Hygiene (naming, verification, idempotence) (10)
